# Task 2


In [ ]:
import os, math, sys, random
import json
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

'cpu'

In [ ]:
HELPER_PATH = "Committor1D_helpers.py"

import importlib.util
spec = importlib.util.spec_from_file_location("comm1d_helpers", HELPER_PATH)
if spec is None or spec.loader is None:
    raise FileNotFoundError(f"Could not load {HELPER_PATH}")
comm = importlib.util.module_from_spec(spec)
sys.modules["comm1d_helpers"] = comm
spec.loader.exec_module(comm)

[name for name in dir(comm) if not name.startswith("_")]

['ChebSum2_Clenshaw_matrix',
 'Cheb_coeffs2',
 'cheb',
 'np',
 'scipy',
 'solve1Dcommittor']

In [ ]:
beta = 1.0

# Discretization for f and q
Nx = 256
x_grid = np.linspace(-1.0, 1.0, Nx).astype(np.float32)

# Chebyshev solver resolution (internal spectral grid)
N_cheb = 256

def Vprime_param(x, p0, p1, p2):
    base = 4*x**3 - 4*x
    gauss = np.exp(-((x - p1)**2) / (2 * p2**2))
    d_bump = p0 * gauss * (-(x - p1) / (p2**2))
    return base + d_bump

# Optional normalizer for the input function (center/scale per-sample)
class PerSampleNormalizer:
    def __init__(self, eps=1e-6):
        self.eps = eps
    def __call__(self, arr):
        # arr: [Nx] numpy
        m = arr.mean()
        s = arr.std()
        if s < self.eps:
            return arr*0.0
        return (arr - m) / (s + self.eps)

In [38]:
def sample_grf_rbf_1d(x, ell=0.5, rng=None):
    """
    Sample u ~ N(0, K) with K_ij = exp(-(x_i - x_j)^2/(2 ell^2)).
    Returns u(x) on the grid x.
    """
    if rng is None:
        rng = np.random.default_rng()
    X = x.reshape(-1,1)
    d2 = (X - X.T)**2
    K = np.exp(-0.5 * d2 / (ell**2))
    # jitter for stability
    K += 1e-6 * np.eye(len(x), dtype=np.float32)
    u = rng.multivariate_normal(mean=np.zeros(len(x)), cov=K).astype(np.float32)
    return u


In [ ]:
class FNOCommittorDataset(Dataset):
    def __init__(self, dataset_type="parametric", Nu=200, Nx=256, seed=42,
                 normalize_input=True, include_stats=True, ell=0.5):
        rng = np.random.default_rng(seed)
        self.Nx = Nx
        self.x = np.linspace(-1.0, 1.0, Nx).astype(np.float32)
        self.normalize_input = normalize_input
        self.include_stats = include_stats
        self.normer = PerSampleNormalizer() if normalize_input else None
        self.ell = ell
        self.dataset_type = dataset_type
        xf_solver = np.linspace(-1.0, 1.0, 1001)
        self.inputs = []   # [Nu, in_channels, Nx]
        self.targets = []  # [Nu, 1, Nx]
        for _ in range(Nu):
            f_fine = self._sample_f(rng, xf_solver)
            qx = comm.solve1Dcommittor(N_cheb, 1.0, 0.0, f_fine, xf_solver, beta, self.x)
            qx = qx.astype(np.float32)
            f_on_grid = np.interp(self.x, xf_solver, f_fine).astype(np.float32)
            mean_raw = float(f_on_grid.mean())
            std_raw = float(f_on_grid.std() + 1e-12)
            channels = [f_on_grid]
            if self.normalize_input:
                channels.append(self.normer(f_on_grid))
            channels.append(self.x)
            if self.include_stats:
                channels.append(np.full((self.Nx,), mean_raw, dtype=np.float32))
                channels.append(np.full((self.Nx,), std_raw, dtype=np.float32))
            inp = np.stack(channels, axis=0).astype(np.float32)
            tar = qx[None, :]
            self.inputs.append(inp)
            self.targets.append(tar)
        self.inputs = torch.from_numpy(np.stack(self.inputs, axis=0))
        self.targets = torch.from_numpy(np.stack(self.targets, axis=0))
    def _sample_f(self, rng, xf_solver):
        if self.dataset_type == "parametric":
            p0 = rng.uniform(0.0, 5.0)
            p1 = rng.uniform(-0.5, 0.5)
            p2 = rng.uniform(0.25, 0.7)
            return Vprime_param(xf_solver, p0, p1, p2).astype(np.float32)
        if self.dataset_type == "grf":
            return sample_grf_rbf_1d(xf_solver, ell=self.ell, rng=rng).astype(np.float32)
        raise ValueError("dataset_type must be 'parametric' or 'grf'")
    def __len__(self):
        return self.inputs.shape[0]
    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

In [ ]:
dataset_type = "parametric" 
Nu_total = 600                # total samples
Nu_train = int(0.8 * Nu_total)
Nu_test  = Nu_total - Nu_train

full_ds = FNOCommittorDataset(dataset_type=dataset_type, Nu=Nu_total, Nx=Nx, seed=SEED, normalize_input=True)
train_ds, test_ds = torch.utils.data.random_split(full_ds, [Nu_train, Nu_test], generator=torch.Generator().manual_seed(SEED))

batch = 32
train_loader = DataLoader(train_ds, batch_size=batch, shuffle=True, drop_last=False)
test_loader  = DataLoader(test_ds,  batch_size=batch, shuffle=False, drop_last=False)

len(train_ds), len(test_ds), next(iter(train_loader))[0].shape

(480, 120, torch.Size([32, 2, 256]))

In [ ]:
class SpectralConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, modes):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.modes = modes 
        self.scale = 1 / (in_channels * out_channels)
        self.weights = nn.Parameter(self.scale * torch.randn(in_channels, out_channels, modes, dtype=torch.cfloat))

    def compl_mul1d(self, a, b):
        return torch.einsum("bim, iom -> bom", a, b)

    def forward(self, x):
        b, c, n = x.shape
        x_ft = torch.fft.rfft(x, dim=-1)  # [b, c, n//2+1]

        # Truncate to low modes
        n_modes = min(self.modes, x_ft.shape[-1])
        out_ft = torch.zeros(b, self.out_channels, x_ft.shape[-1], dtype=torch.cfloat, device=x.device)
        out_ft[:, :, :n_modes] = self.compl_mul1d(x_ft[:, :, :n_modes], self.weights[:, :, :n_modes])
        x_out = torch.fft.irfft(out_ft, n=n, dim=-1)  # [b, out_c, n]
        return x_out

In [ ]:
class FNO1d(nn.Module):
    def __init__(self, in_channels=2, out_channels=1, width=64, modes=32, layers=4, act=nn.GELU):
        super().__init__()
        self.width = width
        self.modes = modes
        self.layers = layers

        self.input_proj  = nn.Conv1d(in_channels, width, kernel_size=1)
        self.spec_convs  = nn.ModuleList([SpectralConv1d(width, width, modes) for _ in range(layers)])
        self.point_convs = nn.ModuleList([nn.Conv1d(width, width, kernel_size=1) for _ in range(layers)])
        self.act = act()

        self.output_proj = nn.Sequential(
            nn.Conv1d(width, width, kernel_size=1),
            self.act,
            nn.Conv1d(width, out_channels, kernel_size=1),
            nn.Sigmoid()  # keep q in [0,1]
        )

    def forward(self, x):
        x = self.input_proj(x)
        for k in range(self.layers):
            x1 = self.spec_convs[k](x)
            x2 = self.point_convs[k](x)
            x  = self.act(x1 + x2)
        out = self.output_proj(x)  # [b, 1, n]
        return out

In [42]:
@torch.no_grad()
def rmse_1d(model, loader, device=DEVICE):
    model.eval()
    se = 0.0
    n  = 0
    for xin, yout in loader:
        xin  = xin.to(device)         # [B, 2, Nx]
        yout = yout.to(device)        # [B, 1, Nx]
        pred = model(xin)
        se += torch.sum((pred - yout)**2).item()
        n  += yout.numel()
    return math.sqrt(se / n)

def train_fno(model, train_loader, test_loader, epochs=200, lr=1e-3, wd=1e-6, bc_weight=1e-2):
    model = model.to(DEVICE)
    opt = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=wd)
    loss_fn = nn.MSELoss()

    best_te = float('inf')
    best = None
    history = []
    for ep in range(1, epochs+1):
        model.train()
        for xin, yout in train_loader:
            xin, yout = xin.to(DEVICE), yout.to(DEVICE)
            opt.zero_grad()
            pred = model(xin)
            data_loss = loss_fn(pred, yout)
            if bc_weight is not None and bc_weight > 0:
                bc_loss = ((pred[:, :, 0])**2 + (pred[:, :, -1] - 1.0)**2).mean()
                loss = data_loss + bc_weight * bc_loss
            else:
                loss = data_loss
            loss.backward()
            opt.step()
        if ep == 1 or ep % 10 == 0 or ep == epochs:
            tr = rmse_1d(model, train_loader, DEVICE)
            te = rmse_1d(model, test_loader, DEVICE)
            history.append((ep, tr, te))
            print(f"Epoch {ep:4d} | Train RMSE {tr:.4e} | Test RMSE {te:.4e}")
            if te < best_te - 1e-6:
                best_te = te
                best = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
    if best is not None:
        model.load_state_dict(best)
        print(f"Restored best checkpoint with Test RMSE = {best_te:.4e}")
    return model, pd.DataFrame(history, columns=["epoch","train_rmse","test_rmse"])


In [ ]:
width = 96
modes = 48  
layers = 4
epochs = 350
lr = 1e-3

model = FNO1d(in_channels=2, out_channels=1, width=width, modes=modes, layers=layers, act=nn.GELU)
model, hist_df = train_fno(model, train_loader, test_loader, epochs=epochs, lr=lr)
hist_df.tail(10)

Epoch    1 | Train RMSE 9.3436e-02 | Test RMSE 9.2405e-02
Epoch   10 | Train RMSE 4.1238e-02 | Test RMSE 4.2023e-02
Epoch   20 | Train RMSE 2.6743e-02 | Test RMSE 2.6156e-02
Epoch   30 | Train RMSE 1.3361e-02 | Test RMSE 1.4370e-02
Epoch   40 | Train RMSE 1.0434e-02 | Test RMSE 1.1187e-02
Epoch   50 | Train RMSE 1.0347e-02 | Test RMSE 1.1165e-02
Epoch   60 | Train RMSE 9.1853e-03 | Test RMSE 9.0941e-03
Epoch   70 | Train RMSE 9.6799e-03 | Test RMSE 9.9465e-03
Epoch   80 | Train RMSE 1.2779e-02 | Test RMSE 1.1479e-02
Epoch   90 | Train RMSE 7.2870e-03 | Test RMSE 7.5912e-03
Epoch  100 | Train RMSE 7.8742e-03 | Test RMSE 8.0643e-03
Epoch  110 | Train RMSE 1.0644e-02 | Test RMSE 1.0495e-02
Epoch  120 | Train RMSE 8.6844e-03 | Test RMSE 9.2757e-03
Epoch  130 | Train RMSE 6.1156e-03 | Test RMSE 6.5986e-03
Epoch  140 | Train RMSE 6.5041e-03 | Test RMSE 6.9949e-03
Epoch  150 | Train RMSE 9.4442e-03 | Test RMSE 1.0176e-02
Epoch  160 | Train RMSE 5.0120e-03 | Test RMSE 5.6136e-03
Epoch  170 | T

,epoch,train_rmse,test_rmse
26,260,0.010120,0.010467
27,270,0.006761,0.006960
28,280,0.004228,0.004857
29,290,0.004628,0.004835
30,300,0.004977,0.005266
31,310,0.009544,0.008928
32,320,0.004862,0.005086
33,330,0.004285,0.004770
34,340,0.005071,0.005421
35,350,0.010219,0.010558


In [ ]:
@torch.no_grad()
def predict_sample(model, xin_np):
    xt = torch.from_numpy(xin_np[None, ...]).to(DEVICE)  # [1, 2, Nx]
    yp = model(xt).cpu().numpy()[0,0]                    # [Nx]
    return yp

# Grab one test sample
xin, ytrue = next(iter(test_loader))
xin_np = xin[0].numpy()          # [2, Nx]
ytrue_np = ytrue[0,0].numpy()    # [Nx]
ypred_np = predict_sample(model, xin_np)

pd.DataFrame({
    "x": x_grid,
    "q_true": ytrue_np,
    "q_pred": ypred_np
}).head()

,x,q_true,q_pred
0,-1.000000,-8.014422e-16,0.002369
1,-0.992157,7.056320e-05,0.001860
2,-0.984314,1.411750e-04,0.001829
3,-0.976471,2.118700e-04,0.002062
4,-0.968627,2.826823e-04,0.002426


In [ ]:
parametric_train_rmse = rmse_1d(model, train_loader, DEVICE)
parametric_test_rmse = rmse_1d(model, test_loader, DEVICE)
fno_parametric_metrics = {
    "dataset": "parametric",
    "train_rmse": float(parametric_train_rmse),
    "test_rmse": float(parametric_test_rmse),
}
fno_parametric_metrics

{'dataset': 'parametric',
 'train_rmse': 0.00428501348628409,
 'test_rmse': 0.004770399145751788}

### GRF


In [ ]:
dataset_type_grf = "grf"
Nu_total_grf = 600
Nu_train_grf = int(0.8 * Nu_total_grf)
Nu_test_grf = Nu_total_grf - Nu_train_grf

full_ds_grf = FNOCommittorDataset(dataset_type=dataset_type_grf, Nu=Nu_total_grf, Nx=Nx, seed=SEED, normalize_input=False, include_stats=True)
train_ds_grf, test_ds_grf = torch.utils.data.random_split(
    full_ds_grf, [Nu_train_grf, Nu_test_grf], generator=torch.Generator().manual_seed(SEED)
)

train_loader_grf = DataLoader(train_ds_grf, batch_size=batch, shuffle=True, drop_last=False)
test_loader_grf = DataLoader(test_ds_grf, batch_size=batch, shuffle=False, drop_last=False)

len(train_ds_grf), len(test_ds_grf), next(iter(train_loader_grf))[0].shape

(480, 120, torch.Size([32, 4, 256]))

In [ ]:
model_grf = FNO1d(in_channels=4, out_channels=1, width=width, modes=modes, layers=layers, act=nn.GELU)
model_grf, hist_df_grf = train_fno(model_grf, train_loader_grf, test_loader_grf, epochs=epochs, lr=lr, bc_weight=1e-2)
hist_df_grf.tail(10)

Epoch    1 | Train RMSE 1.1146e-01 | Test RMSE 1.1053e-01
Epoch   10 | Train RMSE 5.6920e-03 | Test RMSE 5.4223e-03
Epoch   20 | Train RMSE 3.1861e-03 | Test RMSE 3.1856e-03
Epoch   30 | Train RMSE 2.2691e-03 | Test RMSE 2.3637e-03
Epoch   40 | Train RMSE 2.0375e-03 | Test RMSE 2.1414e-03
Epoch   50 | Train RMSE 1.7360e-03 | Test RMSE 2.0247e-03
Epoch   60 | Train RMSE 4.5235e-03 | Test RMSE 4.7191e-03
Epoch   70 | Train RMSE 1.7807e-03 | Test RMSE 1.9348e-03
Epoch   80 | Train RMSE 2.2724e-03 | Test RMSE 2.3896e-03
Epoch   90 | Train RMSE 2.4487e-03 | Test RMSE 2.5209e-03
Epoch  100 | Train RMSE 4.2501e-03 | Test RMSE 4.1510e-03
Epoch  110 | Train RMSE 1.4136e-03 | Test RMSE 1.4409e-03
Epoch  120 | Train RMSE 1.1252e-03 | Test RMSE 1.2476e-03
Epoch  130 | Train RMSE 2.7217e-03 | Test RMSE 2.7329e-03
Epoch  140 | Train RMSE 3.5097e-03 | Test RMSE 3.6682e-03
Epoch  150 | Train RMSE 3.7941e-03 | Test RMSE 4.0109e-03
Epoch  160 | Train RMSE 2.6091e-03 | Test RMSE 2.7661e-03
Epoch  170 | T

,epoch,train_rmse,test_rmse
26,260,0.001445,0.001612
27,270,0.005220,0.005012
28,280,0.002454,0.002886
29,290,0.002254,0.002339
30,300,0.001294,0.001572
31,310,0.002244,0.002605
32,320,0.001389,0.001627
33,330,0.001195,0.001417
34,340,0.001275,0.001542
35,350,0.003211,0.003470


In [ ]:
grf_train_rmse = rmse_1d(model_grf, train_loader_grf, DEVICE)
grf_test_rmse = rmse_1d(model_grf, test_loader_grf, DEVICE)
fno_grf_metrics = {
    "dataset": "grf",
    "train_rmse": float(grf_train_rmse),
    "test_rmse": float(grf_test_rmse),
}
fno_grf_metrics

{'dataset': 'grf',
 'train_rmse': 0.0010921967547413143,
 'test_rmse': 0.0011678126284557914}

In [ ]:
fno_results = {
    "parametric": {
        "train_rmse": fno_parametric_metrics["train_rmse"],
        "test_rmse": fno_parametric_metrics["test_rmse"],
    },
    "grf": {
        "train_rmse": fno_grf_metrics["train_rmse"],
        "test_rmse": fno_grf_metrics["test_rmse"],
    },
}

with open("fno_results.json", "w") as f:
    json.dump(fno_results, f, indent=2)

pd.DataFrame([
    {"model": "FNO", **fno_parametric_metrics},
    {"model": "FNO", **fno_grf_metrics},
])

,model,dataset,train_rmse,test_rmse
0,FNO,parametric,0.004285,0.004770
1,FNO,grf,0.001092,0.001168
